In [0]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import os

In [0]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 307 (delta 33), reused 62 (delta 22), pack-reused 229
Receiving objects: 100% (307/307), 186.89 KiB | 3.81 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [0]:
BUCKET = 'gs://gs_colab' #@param {type:"string"}
PROJECT_NAME = 'tf_flowers'
MODEL_NAME = 'InceptionResNetV2'

BATCH_SIZE = 128 #@param {type: "integer"}
EPOCHS = 20 #@param {type:"integer"}


In [0]:
fw.colab_tpu.setup_gcs()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
data_dir, work_dir, model_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT_NAME, MODEL_NAME)

In [0]:
base_model_func, weight_dir, weight_vars, img_size, normalizer = fw.keras_models.get_inception_resnet_v2(model_dir)

In [0]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.FLOWER_PHOTOS, './flower_photos')

228818944/228813984 [==============================] - 4s 0us/step


In [0]:
# todo: show a random photo from each subdir

In [0]:
data_dir_local = os.path.join(data_dir_local, 'flower_photos')
data_dir_local

'./flower_photos/datasets/flower_photos'

In [0]:
data_fn = os.path.join(data_dir, 'all.tfrecord')

paths, y, labels = fw.io.data_dir_tfrecord(data_dir_local, data_fn, shuffle=True)
len(y), len(labels)

(3670, 5)

In [0]:
class TransferLearningNet(tf.keras.Model):
  def __init__(self, base_model_func, num_cls):
    super().__init__()
    self.base_model = base_model_func()
    self.flatten = tf.keras.layers.Flatten()
    self.linear = tf.keras.layers.Dense(num_cls, use_bias=False)

  def call(self, x):
    return self.linear(self.flatten(self.base_model(x)))

In [0]:
val_sz = len(y) // 5 // 8 * 8
trn_sz = len(y) - val_sz
total_steps = trn_sz // BATCH_SIZE * EPOCHS

In [0]:
opt_func = fw.train.adam_sgdr_one_cycle(total_steps)
get_model = lambda: TransferLearningNet(base_model_func, len(labels))
model_func = fw.tpuest.get_clf_model_func(get_model, opt_func)

In [0]:
parser_train = fw.io.get_tfexample_image_parser(img_size, img_size, training=True, normalizer=normalizer)
parser_eval = fw.io.get_tfexample_image_parser(img_size, img_size, training=False, normalizer=normalizer)


est = fw.tpuest.get_tpu_estimator(trn_sz, val_sz, model_func, work_dir, 
                             weight_dir, weight_vars, BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/tf_flowers', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.24.56.122:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f26850509b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.24.56.122:8470', '_evaluation_master': 'grpc://10.24.56.122:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=22, num_shards=None, num_cores_per_rep

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 0, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 0, training=False)

est.train(train_input_func, steps=total_steps)
result0 = est.evaluate(input_fn=valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Querying Tensorflow master (grpc://10.24.56.122:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 10818251253360784282)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16249077023186077984)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 16652976498145008065)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8642183256017064411)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14184269485812871187)
INFO:tensorflow:*** Available Device: _DeviceAttribute

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 1, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 1, training=False)

est.train(train_input_func, steps=total_steps)
result1 = est.evaluate(input_fn=valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/InceptionResNetV2/keras', vars_to_warm_start=['conv_7b_bn/moving_variance[^/]', 'conv_7b_bn/beta[^/]', 'conv_7b/kernel[^/]', 'conv2d_99/kernel[^/]', 'conv2d_95/kernel[^/]', 'conv2d_94/kernel[^/]', 'conv2d_93/kernel[^/]', 'conv2d_92/kernel[^/]', 'conv2d_9/kernel[^/]', 'conv2d_88/kernel[^/]', 'conv2d_87/kernel[^/]', 'conv_7b_bn/moving_mean[^/]', 'conv2d_86/kernel[^/]', 'conv2d_84/kernel[^/]', 'conv2d_82/kernel[^/]', 'conv2d_81/kernel[^/]', 'conv2d_74/kernel[^/]', 'conv2d_73/kernel[^/]', 'conv2d_71/kernel[^/]', 'conv2d_70/kernel[^/]', 'conv2d_7/kernel[^/]', 'conv2d_68/kernel[^/]', 'conv2d_66/kernel[^/]', 'conv2d_65/kernel

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 2, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 2, training=False)

est.train(train_input_func, steps=total_steps)
result2 = est.evaluate(input_fn=valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/InceptionResNetV2/keras', vars_to_warm_start=['conv_7b_bn/moving_variance[^/]', 'conv_7b_bn/beta[^/]', 'conv_7b/kernel[^/]', 'conv2d_99/kernel[^/]', 'conv2d_95/kernel[^/]', 'conv2d_94/kernel[^/]', 'conv2d_93/kernel[^/]', 'conv2d_92/kernel[^/]', 'conv2d_9/kernel[^/]', 'conv2d_88/kernel[^/]', 'conv2d_87/kernel[^/]', 'conv_7b_bn/moving_mean[^/]', 'conv2d_86/kernel[^/]', 'conv2d_84/kernel[^/]', 'conv2d_82/kernel[^/]', 'conv2d_81/kernel[^/]', 'conv2d_74/kernel[^/]', 'conv2d_73/kernel[^/]', 'conv2d_71/kernel[^/]', 'conv2d_70/kernel[^/]', 'conv2d_7/kernel[^/]', 'conv2d_68/kernel[^/]', 'conv2d_66/kernel[^/]', 'conv2d_65/kernel

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 3, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 3, training=False)

est.train(train_input_func, steps=total_steps)
result3 = est.evaluate(input_fn=valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/InceptionResNetV2/keras', vars_to_warm_start=['conv_7b_bn/moving_variance[^/]', 'conv_7b_bn/beta[^/]', 'conv_7b/kernel[^/]', 'conv2d_99/kernel[^/]', 'conv2d_95/kernel[^/]', 'conv2d_94/kernel[^/]', 'conv2d_93/kernel[^/]', 'conv2d_92/kernel[^/]', 'conv2d_9/kernel[^/]', 'conv2d_88/kernel[^/]', 'conv2d_87/kernel[^/]', 'conv_7b_bn/moving_mean[^/]', 'conv2d_86/kernel[^/]', 'conv2d_84/kernel[^/]', 'conv2d_82/kernel[^/]', 'conv2d_81/kernel[^/]', 'conv2d_74/kernel[^/]', 'conv2d_73/kernel[^/]', 'conv2d_71/kernel[^/]', 'conv2d_70/kernel[^/]', 'conv2d_7/kernel[^/]', 'conv2d_68/kernel[^/]', 'conv2d_66/kernel[^/]', 'conv2d_65/kernel

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 4, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 4, training=False)

est.train(train_input_func, steps=total_steps)
result4 = est.evaluate(input_fn=valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/InceptionResNetV2/keras', vars_to_warm_start=['conv_7b_bn/moving_variance[^/]', 'conv_7b_bn/beta[^/]', 'conv_7b/kernel[^/]', 'conv2d_99/kernel[^/]', 'conv2d_95/kernel[^/]', 'conv2d_94/kernel[^/]', 'conv2d_93/kernel[^/]', 'conv2d_92/kernel[^/]', 'conv2d_9/kernel[^/]', 'conv2d_88/kernel[^/]', 'conv2d_87/kernel[^/]', 'conv_7b_bn/moving_mean[^/]', 'conv2d_86/kernel[^/]', 'conv2d_84/kernel[^/]', 'conv2d_82/kernel[^/]', 'conv2d_81/kernel[^/]', 'conv2d_74/kernel[^/]', 'conv2d_73/kernel[^/]', 'conv2d_71/kernel[^/]', 'conv2d_70/kernel[^/]', 'conv2d_7/kernel[^/]', 'conv2d_68/kernel[^/]', 'conv2d_66/kernel[^/]', 'conv2d_65/kernel

In [0]:
result0, result1, result2, result3, result4

({'accuracy': 0.9629121, 'global_step': 440, 'loss': 0.15541881},
 {'accuracy': 0.96428573, 'global_step': 440, 'loss': 0.1957123},
 {'accuracy': 0.9587912, 'global_step': 440, 'loss': 0.13829023},
 {'accuracy': 0.9478022, 'global_step': 440, 'loss': 0.21208255},
 {'accuracy': 0.9505494, 'global_step': 440, 'loss': 0.1358773})